In [3]:
from mxnet.gluon import nn

def vgg_block(num_convs,channels):
    out=nn.Sequential()
    for _ in range(num_convs):
        out.add(
        nn.Conv2D(channels=channels,kernel_size=3,padding=1,activation='relu')
        )
        out.add(nn.MaxPool2D(pool_size=2,strides=2))
        return out

    

In [8]:
from mxnet import ndarray as nd
#test vgg_block
blk=vgg_block(2,128)
blk.initialize()
x=nd.random.uniform(shape=(2,3,16,16))
y=blk(x)
y.shape
#长宽会减半，通道也会改变。

(2, 128, 8, 8)

In [21]:
#定义如何将这些块堆起来
#没有看懂architecture定义的是什么？按道理architecture作用应该是把vgg_stack(num_convs,channels)循环一下下
#architecture = ((1,64), (1,128), (2,256), (2,512), (2,512))这个定义的是什么，（卷积层，通道数）？
#
def vgg_stack(architecture):
    out = nn.Sequential()
    for (num_convs, channels) in architecture:
        out.add(vgg_block(num_convs, channels))
    return out
            

In [22]:
#他这个含义就是把卷积层全部堆积在一个层层里，叫vgg_stack(architecture)
#接下来该干嘛就干嘛，接Flatten，接全连接层
#VGG 11 8个卷积层，3个全连接层

num_outputs = 10
architecture = ((1,64), (1,128), (2,256), (2,512), (2,512))
net = nn.Sequential()
# add name_scope on the outermost Sequential
with net.name_scope():
    net.add(
        vgg_stack(architecture),
        nn.Flatten(),
        nn.Dense(4096, activation="relu"),
        nn.Dropout(.5),
        nn.Dense(4096, activation="relu"),
        nn.Dropout(.5),
        nn.Dense(num_outputs))
    

In [23]:
import sys
sys.path.append('..')
import utils
from mxnet import gluon
from mxnet import init

train_data,test_data=utils.load_data_fashion_mnist(batch_size=64,resize=96)
ctx = utils.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())
loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.05})
utils.train(train_data, test_data, net, loss,
            trainer, ctx, num_epochs=1)

Start training on  cpu(0)
Epoch 0. Loss: 0.762, Train acc 0.71, Test acc 0.84, Time 2992.3 sec
